In [1]:
# Point 5: Deeper analysis of the data

import pandas as pd
import numpy as np

top_2_array = []
top_5_array = []
top_10_array = []

data = pd.read_csv('data/x_train_gr_smpl.csv')
for i in range(10):
    data = data.reindex(np.arange(data.shape[0]))
    labels = pd.read_csv(f'data/y_train_smpl_{i}.csv')
    data['label'] = labels
    data = data.sample(frac=1)
    
    corr_label = data.drop("label", axis=1).apply(lambda x: x.corr(data.label))
    corr_label = [(index, abs(corr_val)) for index, corr_val in enumerate(corr_label)]
    corr_label = sorted(corr_label, key=lambda tup: tup[1], reverse=True)  # Order by correlation value
    
    for tup in corr_label[:2]:
        if tup[0] not in top_2_array:
            top_2_array.append(tup[0])
    for tup in corr_label[:5]:
        if tup[0] not in top_5_array:
            top_5_array.append(tup[0])
    for tup in corr_label[:10]:
        if tup[0] not in top_10_array:
            top_10_array.append(tup[0])
            

In [2]:
top_2_array.sort()
top_5_array.sort()
top_10_array.sort()

In [3]:
print(len(top_2_array))
print(top_2_array)

20
[1046, 1084, 1094, 1120, 1132, 1168, 1171, 1172, 1186, 1232, 1280, 1319, 1362, 1367, 1410, 1696, 1697, 1738, 1849, 1850]


In [4]:
print(len(top_5_array))
print(top_5_array)

49
[998, 1046, 1082, 1083, 1084, 1094, 1119, 1120, 1131, 1132, 1142, 1167, 1168, 1169, 1171, 1172, 1184, 1186, 1220, 1232, 1270, 1271, 1280, 1319, 1328, 1362, 1363, 1364, 1367, 1368, 1375, 1410, 1411, 1468, 1472, 1648, 1649, 1695, 1696, 1697, 1737, 1738, 1786, 1801, 1848, 1849, 1850, 1897, 1934]


In [5]:
print(len(top_10_array))
print(top_10_array)

98
[981, 997, 998, 1035, 1036, 1043, 1045, 1046, 1071, 1072, 1081, 1082, 1083, 1084, 1094, 1119, 1120, 1121, 1123, 1130, 1131, 1132, 1142, 1167, 1168, 1169, 1171, 1172, 1173, 1179, 1184, 1186, 1190, 1215, 1216, 1220, 1221, 1222, 1232, 1270, 1271, 1280, 1317, 1318, 1319, 1320, 1327, 1328, 1362, 1363, 1364, 1365, 1367, 1368, 1375, 1376, 1410, 1411, 1412, 1413, 1415, 1416, 1419, 1422, 1423, 1468, 1469, 1470, 1472, 1519, 1648, 1649, 1688, 1689, 1695, 1696, 1697, 1698, 1712, 1713, 1737, 1738, 1744, 1745, 1786, 1787, 1800, 1801, 1802, 1848, 1849, 1850, 1885, 1896, 1897, 1898, 1934, 1983]


In [6]:
data = pd.read_csv('data/x_train_gr_smpl.csv')
labels = pd.read_csv('data/y_train_smpl.csv')

print(data[data.columns[top_2_array]])

       1046  1084  1094  1120  1132  1168  1171  1172  1186  1232  1280  1319  \
0      33.0  58.0  34.0  57.0  65.0  63.0  79.0  61.0  66.0  59.0  75.0  58.0   
1      35.0  85.0  34.0  79.0  70.0  74.0  38.0  56.0  67.0  61.0  63.0  51.0   
2      37.0  74.0  37.0  80.0  80.0  82.0  56.0  42.0  65.0  60.0  60.0  56.0   
3      32.0  75.0  36.0  79.0  65.0  80.0  47.0  44.0  60.0  59.0  58.0  45.0   
4      33.0  54.0  31.0  71.0  62.0  75.0  57.0  74.0  39.0  69.0  72.0  62.0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
12655  31.0  50.0  30.0  59.0  43.0  61.0  35.0  36.0  32.0  33.0  32.0  64.0   
12656  31.0  33.0  31.0  35.0  34.0  35.0  43.0  54.0  33.0  32.0  33.0  41.0   
12657  31.0  36.0  32.0  48.0  34.0  50.0  34.0  40.0  33.0  33.0  32.0  54.0   
12658  27.0  34.0  29.0  35.0  34.0  35.0  44.0  56.0  33.0  32.0  33.0  38.0   
12659  28.0  33.0  28.0  34.0  32.0  34.0  45.0  57.0  31.0  31.0  33.0  37.0   

       1362  1367  1410  16